Results for:

In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import FastGradientMethod as FGM

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
import os
path = os.path.join(os.getcwd(), '..','observation_masks.csv')
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [13]:
%%time
kwargs = dict(norm=np.inf,
        eps=0.05,
        num_random_init=5,
        )
attack = utils.define_attack(agent, FGM, **kwargs)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,
                                                        mask=observation_masks['mask_time'].to_numpy().astype(float))

CPU times: total: 4min 19s
Wall time: 4min 18s


In [14]:
kpi

cost_function
annual_peak_average                      1.198347
carbon_emissions_total                   0.897397
cost_total                               0.818383
daily_one_minus_load_factor_average      1.025554
daily_peak_average                       0.960590
electricity_consumption_total            0.905525
monthly_one_minus_load_factor_average    0.989866
ramping_average                          1.251202
zero_net_energy                          1.108729
Name: District, dtype: float64

In [ ]:
pd.DataFrame(obs).to_csv('fgm results/clean obs.csv')
pd.DataFrame(adv_obs).to_csv('fgm results/bb obs.csv')
pd.DataFrame(kpi).to_csv('fgm results/bb kpi.csv')